<center>

# Trabajo Práctico Final

## Procesamiento de imágenes biomédicas

### Primer Cuatrimestre 2024

</center>

**Docentes:**

- Roberto Sebastián Tomás
- Aylin Agatha Vazquez Chenlo
- Francisco Tassara

**Alumnxs:**

- Bajle, Ivo - 62175
- Grau, Gonzalo - 62259
- Pereira, Camila Noemi - 61574

### Objetivo del trabajo

El objetivo del presente trabajo es desarrollar un algoritmo para investigar la translucencia nucal en la etapa gestacional a partir de imágenes de ultrasonido. Este proyecto se llevará a cabo utilizando los conocimientos adquiridos en las clases teóricas de la materia. Al crear un algoritmo capaz de medir la translucencia nucal con la mayor precisión posible, se pretende obtener una comprensión más profunda y proporcionar herramientas precisas para el diagnóstico temprano de anomalías cromosómicas y otros problemas fetales. Los resultados de este estudio podrían contribuir significativamente al campo de la obstetricia. 

### Estado del arte 

En 1985, se descubrió la conexión entre el incremento del pliegue nucal en fetos del segundo trimestre y las alteraciones cromosómicas. En el primer trimestre, se observó una asociación similar y se introdujo el término “translucencia nucal” (TN) para describir la región sonoluscente en la parte posterior de la nuca fetal. Estudios realizados en gestantes de alto riesgo confirmaron esta relación. 

Inicialmente, la TN se evaluaba utilizando un punto de corte fijo. Sin embargo, se observó que la TN aumentaba con la edad gestacional, lo que llevó a la adopción de una curva de valores de referencia en función de la longitud cefalocaudal (LCC). Posteriormente, se desarrolló un modelo matemático que combinaba el riesgo asociado a la TN con la edad materna y los marcadores bioquímicos, dando lugar al método de cribado combinado. Este método resultó ser más eficaz y coste-eficiente que el cribado bioquímico en el segundo trimestre. 

En la práctica clínica rutinaria, se ha comprobado que es posible obtener resultados similares, siempre y cuando se respeten estrictamente las técnicas de medición y los controles de calidad.

Entre la semana 11 y 14 de gestación, todos los fetos desarrollan una colección de líquido en la nuca, visible ecográficamente. El incremento de TN se asocia con edema nucal que puede ser producto de diversas patologías. Las teorías propuestas incluyen fallo cardiaco, congestión venosa, alteraciones en la matriz extracelular y desarrollo anormal del sistema linfático. Después de la semana 14, el sistema linfático se desarrolla lo suficiente para drenar cualquier colección de líquido, por lo que la desaparición del incremento TN no debe interpretarse como un hallazgo tranquilizador.

Para medir la TN, se debe seguir la técnica estándar de la Fetal Medicine Foundation (FMF):
- Realizar la medición cuando la LCC del feto esté entre 45 y 84 mm (semanas 11 a 13+6 días).
- Usar un ecógrafo de alta resolución, preferentemente por vía abdominal. Cuando no sea factible se puede optar por la vía transvaginal.
- La medición debe realizarse en la visión sagital del embrión, con el feto en posición natural y la cabeza alineada con la columna.
- La imagen embrionaria debe estar ampliada de manera que incluya solamente la cabeza fetal y la parte superior del tórax.
- La medición debe realizarse en la zona más ancha de la TN y deben tomarse tres mediciones, quedando como válida la mayor de las tres.
- Para evitar confusiones, se debe diferenciar la línea generada por la membrana amniótica de la correspondiente a la piel fetal y evitar que el cordón umbilical se posicione  alrededor del cuello fetal. 



##### Lo anterior lo saque de aca: https://www.elsevier.es/es-revista-clinica-e-investigacion-ginecologia-obstetricia-7-articulo-translucencia-nucal-13074916 

### Base de datos

Para realizar nuestro trabajo, adquirimos las imágenes necesarias disponibles en el siguiente enlace: https://data.mendeley.com/datasets/n2rbrb9t4f/1. Esta base de datos contiene 1528 imágenes de ultrasonido en vista sagital 2D, recolectadas del Shenzhen People's Hospital de 1519 gestantes. Además, incluye 156 imágenes adicionales provenientes del hospital filial de Longhua del Shenzhen People's Hospital.  